In [375]:
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from opmd_viewer import OpenPMDTimeSeries
import pathlib
path = pathlib.Path('/mnt/hdd/data/gpu_run_debug5')

In [ ]:
!rm -r {path}
!/home/dominik/.miniconda3/envs/nbody3.6/bin/python /home/dominik/Code/NBody-MD/nbody/run_nbody.py --config /home/dominik/Code/NBody-MD/config.json

rm: cannot remove '/mnt/hdd/data/gpu_run_debug5': Directory not empty
/home/dominik/.miniconda3/envs/nbody3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)
 56%|▌| 559/1000 [02:27<01:42,  4.29it/s, kinetic_energy=2.26e-17, mean_p=(-6.817867234769268e-40, 7.758262715427098e-40, -1.88079096131566e-40), mean_r=(1.91e-09, 1.9100000000000002e-09, 1.9100000000000002e-09), potential_energy=2.8e-15, std_p=(1.592545343652039e-23, 1.5647081376967427e-23, 1.5892303380512288e-23), std_r=(1.1012069130437693e-09, 1.1013014774626953e-09, 1.101275032104086e-09), temperature=273]

In [377]:
ts = OpenPMDTimeSeries(path)
ts.slider()

FigureCanvasNbAgg()

In [378]:
import matplotlib.collections as mcoll

def colorline(
        x, y, z=None, cmap='copper', norm=plt.Normalize(0.0, 1.0),
        linewidth=3, alpha=1.0, ax=None):
    """
    http://nbviewer.ipython.org/github/dpsanders/matplotlib-examples/blob/master/colorline.ipynb
    http://matplotlib.org/examples/pylab_examples/multicolored_line.html
    Plot a colored line with coordinates x and y
    Optionally specify colors in the array z
    Optionally specify a colormap, a norm function and a line width
    """

    # Default colors equally spaced on [0,1]:
    if z is None:
        z = np.linspace(0.0, 1.0, len(x))

    # Special case if a single number:
    # to check for numerical input -- this is a hack
    if not hasattr(z, "__iter__"):
        z = np.array([z])

    z = np.asarray(z)

    segments = make_segments(x, y)
    lc = mcoll.LineCollection(segments, array=z, cmap=cmap, norm=norm,
                              linewidth=linewidth, alpha=alpha)

    if ax is None:
        ax = plt.gca()
    ax.add_collection(lc)

    return lc

def make_segments(x, y):
    """
    Create list of line segments from x and y coordinates, in the correct format
    for LineCollection: an array of the form numlines x (points per line) x 2 (x
    and y) array
    """

    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    return segments

In [379]:
from opmd_viewer import ParticleTracker
from ipywidgets import interactive
pt = ParticleTracker( ts, iteration=100, select={'x':[0.0015,0.0017], 'y':[0.0012, 0.0015]}, 
                         species='particles', preserve_particle_index=True )

def get_trajectories(ts, pt):
    species = pt.species
    components = ts.avail_record_components[species]
    N_iterations = len(ts.iterations)
    N_particles = pt.N_selected
    trajectories = {key: np.empty( ( N_iterations, N_particles)) for key in components}
    for i in range( N_iterations ):
        current_values = dict(zip(components, ts.get_particle(components, select=pt, iteration=ts.iterations[i], species=species )))
        for component in current_values:
            trajectories[component][i] = current_values[component]

    fig, ax = plt.subplots()
    n_particle = 0
    x, y = trajectories[components[0]], trajectories[components[1]]
    lc = colorline(x, y, cmap='viridis')
    lines = [lc]
    plt.colorbar(lc)

    def f(n_particle, x_quantity, y_quantity):
        lines.pop().remove()
        x, y = trajectories[x_quantity], trajectories[y_quantity]
        lc = colorline(x[:, n_particle], y[:, n_particle], cmap='viridis', ax=ax)
        lines.append(lc)
        ax.set_xlim(x.min(),
               x.max(),
               )
        ax.set_ylim(y.min(),
                   y.max(),
                   )

        ax.set_xlabel(x_quantity)
        ax.set_ylabel(y_quantity)
        plt.tight_layout()


    interactive_plot = interactive(f,  n_particle=(0, N_particles-1),
                                   x_quantity=components,
                                   y_quantity=components)
    return interactive_plot
get_trajectories(ts, pt)

FigureCanvasNbAgg()

interactive(children=(IntSlider(value=9, description='n_particle', max=19), Dropdown(description='x_quantity',…

In [366]:
# ts.h5_files

# import h5py
# h5_files = [h5py.File(f) for f in ts.h5_files[:3]]

# for f in h5_files:
#     f.close()

In [367]:
import pandas as pd
df = pd.read_json(path / 'diagnostic_results.json', dtype=False).T
df["total_energy"] = df["kinetic_energy"] + df["potential_energy"]

from IPython.display import display

unwrap = ['mean_r', 'std_r', 'mean_p', 'std_p']
nested_data = {index: pd.DataFrame(df[index].values.tolist(), index=df.index, columns=['x', 'y', 'z']) for index in unwrap}
directional_values = pd.concat(nested_data).stack().unstack(level=1).T
# directional_values

directional_values['mean_p'].plot()

FigureCanvasNbAgg()

In [368]:
directional_values['std_r'].plot()

FigureCanvasNbAgg()

In [369]:
df.drop(columns=unwrap).drop(columns=['temperature']).plot(logx=True)

FigureCanvasNbAgg()

In [370]:
df.plot(y='temperature')

FigureCanvasNbAgg()